# Julia Types

We'll create a simple parametric type that allows us to fit polynomials to data like `y = ax + bx^2 + ...`. We'll first write a new type and then overload methods for it. Finally, we'll graph the results of our work, which requires that we install `Gadfly`.

## Gadfly

`Gadfly` is the most common package for graphics in Julia, along the lines of `matplotlib` for Python or `ggplot2` for R. However, it is not installed in Julia or JuliaBox by default. The instructions for installation are a little different for JuliaBox than for IJulia (i.e., the local Jupyter version of Julia).

### Installing Gadfly in JuliaBox

To install `Gadfly` in JuliaBox, do the following:

1. Return to the JuliaBox tab
1. On the top menu, click "Console"
1. Enter `julia` to get the Julia prompt
1. Enter `Pkg.add("Gadfly")` (Note: this may take a while)
1. Optionally, enter `using Gadfly` to precompile things (and this may also take a while)
1. You may need to close the Julia kernel and close the JuliaBox tabs and restart.
1. Then return to this script and start in the next cell.

### Installing Gadfly in IJulia

To install `Gadfly` in IJulia, do the following:

1. Enter `Pkg.add("Gadfly")` (Note: this may take a while)
1. Enter `using Gadfly` to precompile things (and this may also take a while)

In [2]:
using Pkg
Pkg.add("Gadfly")

  Updating registry at `C:\Users\razek\.julia\registries\General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...
 Installed Arpack_jll ───────────── v3.5.0+2
 Installed Compat ───────────────── v2.2.0
 Installed PositiveFactorizations ─ v0.2.3
 Installed IterTools ────────────── v1.3.0
 Installed ColorTypes ───────────── v0.8.1
 Installed Contour ──────────────── v0.5.1
 Installed Gadfly ───────────────── v1.1.0
 Installed Requires ─────────────── v0.5.2
 Installed LineSearches ─────────── v7.0.1
 Installed StatsBase ────────────── v0.32.0
 Installed Rmath ────────────────── v0.6.0
 Installed Measures ─────────────── v0.3.1
 Installed DiffRules ────────────── v1.0.0
 Installed PDMats ───────────────── v0.9.11
 Installed Interpolations ───────── v0.12.8
 Installed ArrayInterface ───────── v2.3.1
 Installed OpenBLAS_jll ─────────── v0.3.7+5
 Installed Showoff ──────────────── v0.3.1
 Installed Parameters ───────────── v0.12.0
 Installe

  Building Rmath → `C:\Users\razek\.julia\packages\Rmath\BoBag\deps\build.log`
  Building FFTW ─→ `C:\Users\razek\.julia\packages\FFTW\qqcBj\deps\build.log`


In [3]:
using Gadfly

┌ Info: Precompiling Gadfly [c91e804a-d5a3-530f-b6f0-dfbca275c004]
└ @ Base loading.jl:1273
┌ Warning: Module JSON with build ID 48272299963001 is missing from the cache.
│ This may mean JSON [682c06a0-de6a-54ab-a142-c8b1cf79cde6] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1011
┌ Warning: Module JSON with build ID 48272299963001 is missing from the cache.
│ This may mean JSON [682c06a0-de6a-54ab-a142-c8b1cf79cde6] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1011
┌ Info: Skipping precompilation since __precompile__(false). Importing Gadfly [c91e804a-d5a3-530f-b6f0-dfbca275c004].
└ @ Base loading.jl:1028
┌ Info: Precompiling Compose [a81c6b42-2e10-5240-aca2-a61377ecd94b]
└ @ Base loading.jl:1273
┌ Warning: Module JSON with build ID 48272299963001 is missing from the cache.
│ This may mean JSON [682c06a0-de6a-54ab-a142-c8b1cf79cde6] does not support precompilation but is imported by a module tha

## Define the new type
We'll define a family of types, parametrized by the degree of the polynomial N. As a note, we could define it as `PolynomialModel{T<:AbstractFloat,N}`, but for simplicity we will just use a single parameter and fix everything to Float64.

In [9]:
type PolynomialModel{N}
    x::Array{Float64,1}
    y::Array{Float64,1}
    coeff::Array{Float64,1}
end

UndefVarError: UndefVarError: N not defined

Next, define a constructor that calls the default one. 

In [5]:
PolynomialModel(x::Array{Float64,1},y::Array{Float64,1},N::Int) =
    PolynomialModel{N}(x,y,rand(N+1))

PolynomialModel (generic function with 1 method)

Then define a second constructor so we can use ranges. Note that types are usually wrapped in a module and exported, so it's easier to modify them.

In [6]:
PolynomialModel(x::LinSpace,y::Array{Float64,1},N::Int) = 
    PolynomialModel(collect(x),y,N)

UndefVarError: UndefVarError: LinSpace not defined

## Use the new type

We create some data by getting 100 uniformly distributed points on the X axis and then getting Y scores with a polynomial function and random noise. We'll then call the new function and fit the data with a second degree polynomial (although the data was created with a third degree).

In [ ]:
x = linspace(-6,6,100)
y = x + 2x.^2 - 0.5x.^3 + 12*rand(100)
m = PolynomialModel(x,y,2)

We can clean up the output by adding a method to Base.show. First, explicitly import show and overload call.

In [ ]:
import Base.show, Base.call 

Write a parametric method, so we can access N.

In [ ]:
function show{N}(io::IO,p::PolynomialModel{N})
    print(io,
"Polynomial Model of degree $N
Coefficients : $(m.coeff)"
    )
end

Look at the cleaner output for the polynomial model.

In [ ]:
m

### Overload the call method

In order to evaluate the model, we'll overload the call method so we can write `m(xi)` to get the value of the polynomial at position x<sub>i</sub>.

In [ ]:
function call{N}(m::PolynomialModel{N}, xi::Float64)
    s = zero(Float64) # It's usually better to use zero, so we don't accidentally write 0 (Int)
    for p=0:N
        s += xi^p * m.coeff[p+1]
    end
    s
end

For integer input.

In [ ]:
call{N}(m::PolynomialModel{N}, xi::Int) = call(m,Float64(xi))

For vectorized input.

In [ ]:
call{N}(m::PolynomialModel{N}, xi::Array{Float64,1}) = [ m(xi[i])::Float64 for i=1:length(xi) ] 

And now get the value of the model when x = 1.

In [ ]:
m(1)

## Method for plotting

We can also create a method to plot the model and the observed data. We'll use `Gadfly.plot`.

In [ ]:
function plot(m::PolynomialModel)
    Gadfly.plot(
        layer(x=m.x,y=m.y,Geom.PointGeometry),
        layer(x=m.x,y=m(m.x),Geom.LineGeometry,Theme(default_color=colorant"black"))
    )
end

### Initial plot

And call the method to plot the results. (This can take a moment.)

In [ ]:
plot(m)

But the method doesn't fit our artificial data well, so we can add another method to fit our model to the data. To do this, the method will need to need to solve the equation Y = X<sup>c</sup>.

As a note, the backslash operator \ solves the linear equation using the proper algorithm depending on the structure of X.

In [ ]:
function fit{N}(m::PolynomialModel{N})
    Nd = length(m.y)
    X = zeros(Nd,N+1)
    for p = 0:N
        X[:,p+1] = m.x.^p
    end
    m.coeff = X \ m.y
end

### Final plot

Now we can see how well the new method fits the observed data.

In [ ]:
m = PolynomialModel(x,y,3)
fit(m)
plot(m)